In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!rm -r -d "/content/drive/My Drive/GP/dstl/4_bands_10_classes_5000_Patches/data"
!rm -r -d msk
!rm -r -d weights

In [ ]:
!mkdir -p msk weights

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import pandas as pd
import numpy as np
from shapely.wkt import loads
from shapely.geometry import MultiPolygon, Polygon
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
import os
import shutil
import glob
import re
import zipfile
!pip install tifffile
import tifffile as tiff
import cv2
import random

K.set_image_data_format('channels_first')

In [ ]:
bands_choices = [3,4,20]
chosen_bands = bands_choices[2]

patches_choices = [2500, 5000]
chosen_patches = patches_choices[1]

case_choices = ["esri", ""]
chosen_case = case_choices[1]

path_choices = [f"/content/drive/My Drive/GP/dstl/{chosen_bands}_bands_10_classes_{chosen_patches}_Patches/", 
                f"/content/drive/My Drive/GP/dstl/trial/{chosen_bands}_bands_10_classes_{chosen_patches}_Patches/"
                ]
chosen_path = path_choices[1]

In [ ]:
if chosen_path == path_choices[1]:
    if os.path.exists(chosen_path) and os.path.isdir(chosen_path):
        shutil.rmtree(chosen_path)
        !mkdir -p msk weights

In [ ]:
trainWKT = pd.read_csv('/content/drive/My Drive/GP/dstl/train_wkt_v4.csv') 
gridSizes = pd.read_csv('/content/drive/My Drive/GP/dstl/grid_sizes.csv', names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)

#Convert all image coordinates
def ConvertCoord(Coords, Size, XYcoords):
    Xmax, Ymin = XYcoords
    Height, Width=Size
    Width = 1.0 * Width * Width / (Width + 1)
    Height = 1.0 * Height * Height / (Height + 1)
    Xfinal , Yfinal = Width / Xmax , Height / Ymin
    Coords[:, 0] *= Xfinal
    Coords[:, 1] *= Yfinal
    IntCoords = np.round(Coords).astype(np.int32)
    return IntCoords

#Find maximum coordinates for x and minimun coordinates for y
def GetXY(GridSizes, ImageId):
    Xmax, Ymin = GridSizes[GridSizes.ImageId == ImageId].iloc[0, 1:].astype(float)
    return (Xmax, Ymin)

#Find polygon list for class for image
def GetPolygonsList(wkt_list, ImageId, classType):
    Allpolygon = wkt_list[wkt_list.ImageId == ImageId]
    multipolygon = Allpolygon[Allpolygon.ClassType == classType].MultipolygonWKT
    polygonList = None
    if len(multipolygon) != 0:
        polygonList = loads(multipolygon.values[0])
    return polygonList

def GetTheConvertedContours(PolygonList, Size, XYCoods):
    if PolygonList is None:
        return None
    if Size is None or Size[0] <= 0 or Size[1] <= 0:
        return None
    ExteriorList,InteriorList = [],[]
    for Polygon in PolygonList:
        ExteriorList.append(    ConvertCoord( np.array(list(Polygon.exterior.coords)) , Size, XYCoods)  )
        for j in Polygon.interiors:
            InteriorList.append(ConvertCoord(np.array(list(j.coords)), Size, XYCoods))
    return ExteriorList, InteriorList

def PlotMask(Size, Contours):
    if Size is None:
        return None
    if Size[0] <= 0 or Size[1] <= 0:
        return None
    mask = np.zeros(Size, np.uint8)
    if Contours is None:
        return mask
    ExteriorList, InteriorList = Contours
    cv2.fillPoly(mask, ExteriorList, 1)
    cv2.fillPoly(mask, InteriorList, 0)
    return mask

#Fill polygons exterior and interior points and return mask of images
def GenerateMaskForImage(img_size, imageId, class_type, GridSizes=gridSizes, wktList=trainWKT):
    XY = GetXY(GridSizes, imageId)
    PolygonList = GetPolygonsList(wktList, imageId, class_type)
    contours = GetTheConvertedContours(PolygonList, img_size, XY)
    Mask = PlotMask(img_size, contours)
    return Mask


def ContrastAdjustment(img, low_p = 5, high_p = 95):
    img_adjusted = np.zeros_like(img, dtype=np.float32)
    for i in range(img.shape[2]):
        mn, mx = 0, 1       # np.min(img), np.max(img)
        p_low, p_high = np.percentile(img[:, :, i], low_p), np.percentile(img[:, :, i], high_p)
        tmp = mn + (img[:, :, i] - p_low) * (mx - mn) / (p_high - p_low)
        tmp[tmp < mn] = mn
        tmp[tmp > mx] = mx
        img_adjusted[:, :, i] = tmp
    return img_adjusted.astype(np.float32)
    
def ReadTif(image_id, bands=chosen_bands, size=800, case=chosen_case):
    if bands == 3:
        if case == "esri":
            dir = "/content/drive/My Drive/GP/Raster/"
            pattern = r'*.tif'
            gen = glob.iglob(os.path.join(dir, pattern))
            filename = next(gen)
            img_4 = tiff.imread(filename)
            img = np.zeros((3000, 3000, bands), "float32")
            img = img_4[ 1500:2500 , 5000:6000 , 0:3]
            #img = np.rollaxis(img, 0, 3)
            img = cv2.resize(img, (size, size))
        else:
            filename = "/content/drive/My Drive/GP/dstl/three_band/{}.tif".format(image_id)
            img = tiff.imread(filename)
            img = np.rollaxis(img, 0, 3)
            img = cv2.resize(img, (size, size))

    elif bands == 4:
        if case == 'esri':
            dir = "/content/drive/My Drive/GP/Raster/"
            pattern = r'*.tif'
            gen = glob.iglob(os.path.join(dir, pattern))
            filename = next(gen)
            img = tiff.imread(filename)
            #img = cv2.resize(img, (size, size))
        else:
            # for type M 
            img_M = np.transpose(tiff.imread("/content/drive/My Drive/GP/dstl/sixteen_band/{}_M.tif".format(image_id)), (1,2,0))
            img_M = cv2.resize(img_M, (size, size))

            # for RGB 
            img_RGB = tiff.imread("/content/drive/My Drive/GP/dstl/three_band/{}.tif".format(image_id))
            img_RGB = np.rollaxis(img_RGB, 0, 3)
            img_RGB = cv2.resize(img_RGB, (size, size))

            img = np.zeros((img_RGB.shape[0], img_RGB.shape[1], bands), "float32")
            #print(f'RGB shape: {img_RGB.shape}\nM shape: {img_M.shape}\nimg shape: {img.shape}')
            img[..., 0:3] = img_RGB
            img[..., 3] = img_M[ : , : , 7]
        
    elif bands==20:
        # for type M 
        img_M = np.transpose(tiff.imread("/content/drive/My Drive/GP/dstl/sixteen_band/{}_M.tif".format(image_id)), (1,2,0))
        img_M = cv2.resize(img_M, (size, size))
        # for type A
        img_A = np.transpose(tiff.imread("/content/drive/My Drive/GP/dstl/sixteen_band/{}_A.tif".format(image_id)), (1,2,0))
        img_A = cv2.resize(img_A, (size, size))
        # for type P
        img_P = tiff.imread("/content/drive/My Drive/GP/dstl/sixteen_band/{}_P.tif".format(image_id))
        img_P = cv2.resize(img_P, (size, size))

        filename = "/content/drive/My Drive/GP/dstl/three_band/{}.tif".format(image_id)
        # for RGB 
        img_RGB = tiff.imread(filename)
        img_RGB = np.rollaxis(img_RGB, 0, 3)
        img_RGB = cv2.resize(img_RGB, (size, size))

        img = np.zeros((img_RGB.shape[0], img_RGB.shape[1], bands), "float32")
        img[..., 0:3] = img_RGB
        img[..., 3] = img_P
        img[..., 4:12] = img_M
        img[..., 12:21] = img_A

    return img


In [ ]:
num_classes = 10 # total number of class

def StartTrain():
    #put all image into one image
    Size = 800
    x = np.zeros((5 * Size, 5 * Size, chosen_bands)) #input for Unet
    y = np.zeros((5 * Size, 5 * Size, num_classes)) # exp output
    ids = sorted(trainWKT.ImageId.unique())
    for i in range(5):
        for j in range(5):
            id = ids[5 * i + j]
            img = ReadTif(id)
            img = ContrastAdjustment(img)
            Xi,Yi=Size * i,Size * j
            x[Xi : Xi + Size, Yi : Yi + Size, :] = img[:Size, :Size,:]
            for Cls in range(num_classes):
                y[Xi:Xi + Size, Yi : Yi + Size, Cls] = GenerateMaskForImage((img.shape[0], img.shape[1]),id, Cls + 1) #[:Size, :Size]
    np.save((chosen_path+'data/x_trn_%d') % num_classes, x)
    np.save((chosen_path+'data/y_trn_%d') % num_classes, y)


def generateSamples(img, msk, amt=chosen_patches, aug=True):
    xm, ym = img.shape[0] - InputLayerSize, img.shape[1] - InputLayerSize
    x, y = [], []
    bestThreshold = [0.4, 0.1, 0.1, 0.15, 0.3, 0.95, 0.1, 0.05, 0.001, 0.005]     # from trial and error 
    for i in range(amt):
        x_random = random.randint(0, xm)
        y_random = random.randint(0, ym)
        img_random = img[x_random:x_random + InputLayerSize, y_random:y_random + InputLayerSize]        # random image
        msk_random = msk[x_random:x_random + InputLayerSize, y_random:y_random + InputLayerSize]        # random mask 
        for j in range(num_classes):
            if 1.0 * np.sum(msk_random[:, :, j]) / InputLayerSize ** 2 > bestThreshold[j]:
                if random.uniform(0, 1) > 0.5:
                    img_random = img_random[::-1]
                    msk_random = msk_random[::-1]
                if random.uniform(0, 1) > 0.5:
                    img_random = img_random[:, ::-1]
                    msk_random = msk_random[:, ::-1]
                x.append(img_random)
                y.append(msk_random)
    x, y = 2 * np.transpose(x, (0, 3, 1, 2)) - 1, np.transpose(y, (0, 3, 1, 2))
    return x, y

## Split train and validation data

In [ ]:
def pickValidation():
    # pick samples for validatio
    img = np.load((chosen_path+'data/x_trn_%d.npy') % num_classes)
    msk = np.load((chosen_path+'data/y_trn_%d.npy') % num_classes)
    
    x, y = generateSamples(img, msk, amt=1500)

    np.save((chosen_path+'data/x_tmp_%d') % num_classes, x)
    np.save((chosen_path+'data/y_tmp_%d') % num_classes, y)

In [ ]:
InputLayerSize = 160

def conv_block(n_neuron, input):
    return Conv2D(n_neuron, 3, activation='relu', padding='same')(input)

def maxPool_block(input):
    return MaxPooling2D(pool_size=(2, 2))(input)

def upSample_block(input1, input2):
    return concatenate([UpSampling2D(size=(2, 2))(input1), input2], axis=1)

def buildModel(nStart_neuron = 32):
    input_layer = Input((chosen_bands, InputLayerSize, InputLayerSize))

    # Contracting Path [Down Sampling]

    conv1_1 = conv_block(nStart_neuron, input_layer)
    conv1_2 = conv_block(nStart_neuron, conv1_1)
    max_pool1 = maxPool_block(conv1_2)

    conv2_1 = conv_block(nStart_neuron * 2, max_pool1)
    conv2_2 = conv_block(nStart_neuron * 2, conv2_1)
    max_pool2 = maxPool_block(conv2_2)

    conv3_1 = conv_block(nStart_neuron * 4, max_pool2)
    conv3_2 = conv_block(nStart_neuron * 4, conv3_1)
    max_pool3 = maxPool_block(conv3_2)

    conv4_1 = conv_block(nStart_neuron* 8, max_pool3)
    conv4_2 = conv_block(nStart_neuron* 8, conv4_1)
    max_pool4 = maxPool_block(conv4_2)

    conv5_1 = conv_block(nStart_neuron * 16, max_pool4)
    conv5_2 = conv_block(nStart_neuron * 16, conv5_1)

    # Expansive Path [Up Sampling]

    up_sample6 = upSample_block(conv5_2, conv4_2)
    conv6_1 = conv_block(nStart_neuron * 8, up_sample6)
    conv6_2 = conv_block(nStart_neuron * 8, conv6_1)

    up_sample7 = upSample_block(conv6_2, conv3_2)
    conv7_1 = conv_block(nStart_neuron * 4, up_sample7)
    conv7_2 = conv_block(nStart_neuron * 4, conv7_1)

    up_sample8 = upSample_block(conv7_2, conv2_2)
    conv8_1 = conv_block(nStart_neuron * 2, up_sample8)
    conv8_2 = conv_block(nStart_neuron * 2, conv8_1)

    up_sample9 = upSample_block(conv8_2, conv1_2)
    conv9_1 = conv_block(nStart_neuron, up_sample9)
    conv9_2 = conv_block(nStart_neuron, conv9_1)

    finalConv = Conv2D(num_classes, 1, activation='sigmoid')(conv9_2)

    model = Model(input=input_layer, output=finalConv)
    model.compile(optimizer = Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = buildModel()
print(chosen_path)
if os.path.isfile(f"{chosen_path}unet_{chosen_bands}_10_jk_BestScore"):
    model.load_weights(f"{chosen_path}unet_{chosen_bands}_10_jk_BestScore")
    print("weights loaded ......")
#model.summary()

In [ ]:
import keras
import pydot
from keras.utils import plot_model
import pydotplus
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot
from IPython.display import Image

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
Image(retina=True, filename='model.png');

In [ ]:
def jaccard_similarity_score(Y1, Y2):
    Y = Y1 == Y2
    return np.count_nonzero(Y) / Y.size

def calcAccuracy(model):
    img = np.load((chosen_path+'data/x_tmp_%d.npy') % num_classes)
    msk = np.load((chosen_path+'data/y_tmp_%d.npy') % num_classes)

    prd = model.predict(img, batch_size=4)
    sumValues, thresholds = [], []

    for i in range(num_classes):
        mskTrain, prdTrain = msk[:, i, :, :], prd[:, i, :, :]
        mskTrain = mskTrain.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        prdTrain = prdTrain.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        mx, threshold = 0, 0
        for j in range(10):
            score = jaccard_similarity_score(mskTrain, prdTrain > j / 10.0)      
            if score > mx:
                mx = score
                threshold = j / 10.0
        sumValues.append(mx)
        thresholds.append(threshold)

    accuracy = sum(sumValues) / 10.0
    return accuracy, thresholds

In [ ]:
def trainModel():
    xValidate, yValidate = np.load((chosen_path+'data/x_tmp_%d.npy') % num_classes), np.load((chosen_path+'data/y_tmp_%d.npy') % num_classes)    # load validate data
    imgTrained = np.load((chosen_path+'data/x_trn_%d.npy') % num_classes)
    mskTrained = np.load((chosen_path+'data/y_trn_%d.npy') % num_classes)
  
    model_checkpoint = ModelCheckpoint('weights/unet_tmp.hdf5', monitor='loss', save_best_only=True)   
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs')        
    arr = []
    for _ in range(2):
        xTrain, yTrain = generateSamples(imgTrained, mskTrained) 
        model.fit(xTrain, yTrain, batch_size=64, nb_epoch=5, verbose=1, shuffle=True,
                  callbacks=[model_checkpoint,tensorboard_callback], validation_data=(xValidate, yValidate))       
        del x_trn
        del y_trn
        accuracy, thresholds = calcAccuracy(model)
        arr.append(float("{:.4f}".format(accuracy)))
        model.save_weights((f'{chosen_path}unet_{chosen_bands}_10_jk%.4f') % accuracy)
    arr.sort()
    os.rename(f'{chosen_path}unet_{chosen_bands}_10_jk{arr[-1]}', f'{chosen_path}unet_{chosen_bands}_10_jk_BestScore')
    return model

In [ ]:
if chosen_path == path_choices[1]: StartTrain()

In [ ]:
if chosen_path == path_choices[1]: pickValidation()

In [ ]:
if chosen_path == path_choices[1]: model = trainModel()

In [ ]:
accuracy, thresholds = calcAccuracy(model)

In [ ]:
chosen_bands = 3
chosen_patches = 5000
chosen_path = f"/content/drive/My Drive/GP/dstl/{chosen_bands}_bands_10_classes_{chosen_patches}_Patches/"

model = buildModel()
model.load_weights(f'{chosen_path}unet_{chosen_bands}_10_jk_BestScore')
accuracy, thresholds = calcAccuracy(model)
print(accuracy)
print(thresholds)
np.save(f'{chosen_path}unet_{chosen_bands}_thresholds', thresholds)

In [ ]:
chosen_bands = 4
chosen_patches = 5000
chosen_path = f"/content/drive/My Drive/GP/dstl/{chosen_bands}_bands_10_classes_{chosen_patches}_Patches/"

model = buildModel()
model.load_weights(f'{chosen_path}unet_{chosen_bands}_10_jk_BestScore')
accuracy, thresholds = calcAccuracy(model)
print(accuracy)
print(thresholds)
np.save(f'{chosen_path}unet_{chosen_bands}_thresholds', thresholds)

In [ ]:
chosen_bands = 20
chosen_patches = 2500
chosen_path = f"/content/drive/My Drive/GP/dstl/{chosen_bands}_bands_10_classes_{chosen_patches}_Patches/"

model = buildModel()
model.load_weights(f'{chosen_path}unet_{chosen_bands}_10_jk_BestScore')
accuracy, thresholds = calcAccuracy(model)
print(accuracy)
print(thresholds)

In [ ]:
chosen_bands = 20
chosen_patches = 5000
chosen_path = f"/content/drive/My Drive/GP/dstl/{chosen_bands}_bands_10_classes_{chosen_patches}_Patches/"

model = buildModel()
model.load_weights(f'{chosen_path}unet_{chosen_bands}_10_jk_BestScore')
accuracy, thresholds = calcAccuracy(model)
print(accuracy)
print(thresholds)
np.save(f'{chosen_path}unet_{chosen_bands}_thresholds', thresholds)